## Step 1

In [ ]:
import pandas as pd
import re

# Load the Excel file
df = pd.read_excel("/content/Secod_File.xlsx", sheet_name="Customer and Purchase")

def extract_specific_cases(cell):
    if pd.isna(cell):
        return [None, None, None]

    text = str(cell).upper().strip()

    # --- Case 1: "1+1 / 1 ADET" ---
    match_case1 = re.match(r'^\s*([\d\+]+)\s*/\s*(\d+)\s*ADET\s*$', text)
    if match_case1:
        return [f"{match_case1.group(1)} {match_case1.group(2)}", None, None]

    # --- Case 3: "1+1 / 2 ADT" (ADT instead of ADET) ---
    match_case3 = re.match(r'^\s*([\d\+]+)\s*/\s*(\d+)\s*ADT\s*$', text)
    if match_case3:
        return [f"{match_case3.group(1)} {match_case3.group(2)}", None, None]

    # --- Case 2: multiple matches like "1+1 2 ADET / 1 ADET 1+0 / 1 ADET 2+1" ---
    match_case2 = re.findall(r'([\d\+]+)\s+(\d+)\s*ADET|(\d+)\s*ADET\s*([\d\+]+)', text)
    results = []
    for part in match_case2:
        if part[0] and part[1]:  # e.g., 1+1 2 ADET
            results.append(f"{part[0]} {part[1]}")
        elif part[2] and part[3]:  # e.g., 1 ADET 1+0
            results.append(f"{part[3]} {part[2]}")

    return results[:3] + [None] * (3 - len(results))

# Apply to the column
df[['type a', 'type b', 'type c']] = df['Type of Apartment Purchased'].apply(extract_specific_cases).apply(pd.Series)

# Final dataframe
final_df = df[['Customer Name', 'Type of Apartment Purchased', 'Number of Apartment Purchased', 'type a', 'type b', 'type c']]

# Save the result
final_df.to_excel("Refined_Apartment_type.xlsx", index=False)

print("✅ Saved to 'Refined_Apartment_type.xlsx'")


✅ Saved to 'Refined_Apartment_type.xlsx'


In [ ]:
df = pd.read_excel("/content/Refined_Apartment_type.xlsx")
df

,Customer Name,Type of Apartment Purchased,Number of Apartment Purchased,type a,type b,type c
0,HALİL GEÇİCİ,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
1,NESLİŞAH İSTENCİOĞLU,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
2,OLCAY ORUÇ,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
3,SEHER YALÇIN,1+1 2 ADET / 1 ADET 1+0 /1 ADET 2+1,4.0,1+1 2,1+0 1,2+1 1
4,SEMA AKÇIL,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
...,...,...,...,...,...,...
148,GÜVEN ATAR,1+1 /1 ADET,1.0,1+1 1,NaN,NaN
149,HÜSNÜ AYDINALP,1 ADET1+1 / 1 ADET 2+1,2.0,1+1 1,2+1 1,NaN
150,JAROSLAW WOJCIECH,1 ADET 1+1,NaN,1+1 1,NaN,NaN
151,ROZA JOWITA ORZEL,1 ADET 1+0,NaN,1+0 1,NaN,NaN


# Step 2

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("/content/Refined_Apartment_type.xlsx")

# Fill in '1:1 1' to 'type a' if 'Type of Apartment Purchased' is empty and Number > 0
for index, row in df.iterrows():
    type_of_apartment = row.get("Type of Apartment Purchased")
    num_apartments = row.get("Number of Apartment Purchased")

    # Check condition: Type is empty and Number > 0
    if pd.isna(type_of_apartment) and pd.notna(num_apartments) and num_apartments > 0:
        df.at[index, "type a"] = "1+1 1"

# Save the updated DataFrame
df.to_excel("Refined_Type.xlsx", index=False)

print("✅ Done! File saved as 'Refined_Type.xlsx'")


✅ Done! File saved as 'Refined_Type.xlsx'


In [ ]:
df = pd.read_excel("/content/Refined_Type.xlsx")
df

,Customer Name,Type of Apartment Purchased,Number of Apartment Purchased,type a,type b,type c
0,HALİL GEÇİCİ,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
1,NESLİŞAH İSTENCİOĞLU,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
2,OLCAY ORUÇ,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
3,SEHER YALÇIN,1+1 2 ADET / 1 ADET 1+0 /1 ADET 2+1,4.0,1+1 2,1+0 1,2+1 1
4,SEMA AKÇIL,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
...,...,...,...,...,...,...
148,GÜVEN ATAR,1+1 /1 ADET,1.0,1+1 1,NaN,NaN
149,HÜSNÜ AYDINALP,1 ADET1+1 / 1 ADET 2+1,2.0,1+1 1,2+1 1,NaN
150,JAROSLAW WOJCIECH,1 ADET 1+1,NaN,1+1 1,NaN,NaN
151,ROZA JOWITA ORZEL,1 ADET 1+0,NaN,1+0 1,NaN,NaN


# Step 3

In [ ]:
import pandas as pd
import re

# Load the Excel file
df = pd.read_excel("/content/Refined_Type.xlsx")

# Columns to expand
expand_cols = ['type a', 'type b', 'type c']

# Parse the value and count from cells like '1+1 2'
def parse_value(cell):
    if pd.isna(cell):
        return None, 0
    match = re.match(r"(.+?)\s+(\d+)", str(cell).strip())
    if match:
        return match.group(1).strip(), int(match.group(2))
    else:
        return str(cell).strip(), 1  # If no count, assume 1

expanded_rows = []

for _, row in df.iterrows():
    values = {}
    repeat_counts = []

    # Parse type columns and get counts
    for col in expand_cols:
        val, count = parse_value(row[col])
        values[col] = (val, count)
        repeat_counts.append(count)

    # Ensure at least one repetition even if all counts are zero
    max_repeat = max(repeat_counts) if max(repeat_counts) > 0 else 1

    # Repeat rows accordingly
    for i in range(max_repeat):
        new_row = row.copy()
        for col in expand_cols:
            val, count = values[col]
            if count == 0:
                # Column was empty, keep empty string
                new_row[col] = ''
            else:
                # If current iteration is less than count, put val + ' 1'
                if i < count:
                    new_row[col] = f"{val} 1"
                else:
                    # Beyond count, leave empty
                    new_row[col] = ''
        expanded_rows.append(new_row)

# Create expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# If you want to **add final expanded rows** to original data as well:
# final_df = pd.concat([df, expanded_df], ignore_index=True)
# Otherwise just use expanded_df

expanded_df.to_excel("final_expanded_output.xlsx", index=False)

print("✅ Done! Output saved to 'final_expanded_output.xlsx'")


✅ Done! Output saved to 'final_expanded_output.xlsx'


In [ ]:
df = pd.read_excel("/content/final_expanded_output.xlsx")
df

,Customer Name,Type of Apartment Purchased,Number of Apartment Purchased,type a,type b,type c
0,HALİL GEÇİCİ,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
1,NESLİŞAH İSTENCİOĞLU,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
2,OLCAY ORUÇ,1+1 / 1 ADET,1.0,1+1 1,NaN,NaN
3,SEHER YALÇIN,1+1 2 ADET / 1 ADET 1+0 /1 ADET 2+1,4.0,1+1 1,1+0 1,2+1 1
4,SEHER YALÇIN,1+1 2 ADET / 1 ADET 1+0 /1 ADET 2+1,4.0,1+1 1,NaN,NaN
...,...,...,...,...,...,...
279,GÜVEN ATAR,1+1 /1 ADET,1.0,1+1 1,NaN,NaN
280,HÜSNÜ AYDINALP,1 ADET1+1 / 1 ADET 2+1,2.0,1+1 1,2+1 1,NaN
281,JAROSLAW WOJCIECH,1 ADET 1+1,NaN,1+1 1,NaN,NaN
282,ROZA JOWITA ORZEL,1 ADET 1+0,NaN,1+0 1,NaN,NaN


# Step 4

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("/content/final_expanded_output.xlsx")

# Fill NaN with empty strings
df[['type a', 'type b', 'type c']] = df[['type a', 'type b', 'type c']].fillna('')

# Condition 1: 'type a' == '1+1 1'
cond1 = df['type a'] == '1+1 1'
df_cond1 = df[cond1].copy()
df_cond1['Type of Apartment'] = '1+1 1'
df_cond1['Number of Rooms'] = 1

# Condition 2: ANY of 'type a', 'type b', 'type c' == '1+0 1'
cond2 = (df['type a'] == '1+0 1') | (df['type b'] == '1+0 1') | (df['type c'] == '1+0 1')
df_cond2 = df[cond2].copy()
df_cond2['Type of Apartment'] = '1+0 1'
df_cond2['Number of Rooms'] = 1

# Condition 3: ANY of 'type a', 'type b', 'type c' == '2+1 1'
cond3 = (df['type a'] == '2+1 1') | (df['type b'] == '2+1 1') | (df['type c'] == '2+1 1')
df_cond3 = df[cond3].copy()
df_cond3['Type of Apartment'] = '2+1 1'
df_cond3['Number of Rooms'] = 2

# Keep only required columns
cols_to_keep = ['Customer Name', 'Number of Rooms', 'Type of Apartment']

df_cond1 = df_cond1[cols_to_keep]
df_cond2 = df_cond2[cols_to_keep]
df_cond3 = df_cond3[cols_to_keep]

# Save to Excel files
df_cond1.to_excel("Block_A.xlsx", index=False)
df_cond2.to_excel("Block_B.xlsx", index=False)
df_cond3.to_excel("Block_C.xlsx", index=False)

print("Files saved: type_1.xlsx, type_2.xlsx, and type_3.xlsx with final type and number of rooms.")


Files saved: type_1.xlsx, type_2.xlsx, and type_3.xlsx with final type and number of rooms.


In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("/content/final_expanded_output.xlsx")

# Fill NaN with empty strings
df[['type a', 'type b', 'type c']] = df[['type a', 'type b', 'type c']].fillna('')

# Condition 1: 'type a' == '1+1 1'
cond1 = df['type a'] == '1+1 1'
df_cond1 = df[cond1].copy()
df_cond1['Type of Apartment'] = '1+1 1'
df_cond1['Number of Rooms'] = 1

# Condition 2: ANY of 'type a', 'type b', 'type c' == '1+0 1'
cond2 = (df['type a'] == '1+0 1') | (df['type b'] == '1+0 1') | (df['type c'] == '1+0 1')
df_cond2 = df[cond2].copy()
df_cond2['Type of Apartment'] = '1+0 1'
df_cond2['Number of Rooms'] = 1

# Condition 3: ANY of 'type a', 'type b', 'type c' == '2+1 1'
cond3 = (df['type a'] == '2+1 1') | (df['type b'] == '2+1 1') | (df['type c'] == '2+1 1')
df_cond3 = df[cond3].copy()
df_cond3['Type of Apartment'] = '2+1 1'
df_cond3['Number of Rooms'] = 2

# Add Flat ID Name column (1 to N)
df_cond1['Flat ID Name'] = range(1, len(df_cond1) + 1)
df_cond2['Flat ID Name'] = range(1, len(df_cond2) + 1)
df_cond3['Flat ID Name'] = range(1, len(df_cond3) + 1)

# Keep only required columns
cols_to_keep = ['Flat ID Name', 'Customer Name', 'Number of Rooms', 'Type of Apartment']
df_cond1 = df_cond1[cols_to_keep]
df_cond2 = df_cond2[cols_to_keep]
df_cond3 = df_cond3[cols_to_keep]

# Save to Excel files
df_cond1.to_excel("Block_A.xlsx", index=False)
df_cond2.to_excel("Block_B.xlsx", index=False)
df_cond3.to_excel("Block_C.xlsx", index=False)

print("Files saved: Block_A.xlsx, Block_B.xlsx, and Block_C.xlsx with Flat ID Name added.")


Files saved: Block_A.xlsx, Block_B.xlsx, and Block_C.xlsx with Flat ID Name added.


In [ ]:
df = pd.read_excel("/content/Block_A.xlsx")
df

,Flat ID Name,Customer Name,Number of Rooms,Type of Apartment
0,1,HALİL GEÇİCİ,1,1+1 1
1,2,NESLİŞAH İSTENCİOĞLU,1,1+1 1
2,3,OLCAY ORUÇ,1,1+1 1
3,4,SEHER YALÇIN,1,1+1 1
4,5,SEHER YALÇIN,1,1+1 1
...,...,...,...,...
260,261,ŞENOL ÇAKIR,1,1+1 1
261,262,AMELİA KATAR ZYLA,1,1+1 1
262,263,GÜVEN ATAR,1,1+1 1
263,264,HÜSNÜ AYDINALP,1,1+1 1


In [ ]:
df = pd.read_excel("/content/Block_B.xlsx")
df

,Flat ID Name,Customer Name,Number of Rooms,Type of Apartment
0,1,SEHER YALÇIN,1,1+0 1
1,2,HATİCE OKÇU,1,1+0 1
2,3,HICHAM HALLOU,1,1+0 1
3,4,HICHAM HALLOU,1,1+0 1
4,5,HICHAM HALLOU,1,1+0 1
5,6,HICHAM HALLOU,1,1+0 1
6,7,HICHAM HALLOU,1,1+0 1
7,8,HICHAM HALLOU,1,1+0 1
8,9,İSMAİL HAKKI ÖNDER,1,1+0 1
9,10,SEDA ÇERÇİZ VE ASİYE SERAP KARASULAR,1,1+0 1


In [ ]:
df = pd.read_excel("/content/Block_C.xlsx")
df

,Flat ID Name,Customer Name,Number of Rooms,Type of Apartment
0,1,SEHER YALÇIN,2,2+1 1
1,2,HÜSNÜ AYDINALP,2,2+1 1


# Final Output

In [ ]:
import pandas as pd
import random

# File names
files = ['Block_A.xlsx', 'Block_B.xlsx', 'Block_C.xlsx']
final_output_file = 'Final_output.xlsx'

# Load the 3 Excel files into DataFrames
dfs = [pd.read_excel(f) for f in files]

# Check for required columns
expected_columns = ['Customer Name', 'Number of Rooms', 'Type of Apartment']
for i, df in enumerate(dfs):
    if not all(col in df.columns for col in expected_columns):
        raise ValueError(f"File {files[i]} does not contain required columns.")

# Initialize output DataFrame
final_output = pd.DataFrame(columns=expected_columns)

while any(not df.empty for df in dfs):
    # Filter to only non-empty DataFrames
    non_empty_indices = [i for i, df in enumerate(dfs) if not df.empty]

    # Randomly select one of the non-empty files
    file_idx = random.choice(non_empty_indices)
    df = dfs[file_idx]

    # Randomly select one row from this DataFrame
    idx = random.choice(df.index)
    selected_row = df.loc[idx]

    # Append to final output
    final_output = pd.concat([final_output, selected_row.to_frame().T], ignore_index=True)

    # Remove selected row from source DataFrame
    dfs[file_idx] = df.drop(idx)

    print(f"Selected from {files[file_idx]}: {selected_row.to_dict()}")

# Save updated source files (after removals)
for i, df in enumerate(dfs):
    df.to_excel(files[i], index=False)

# Save final output file
final_output.to_excel(final_output_file, index=False)

print("Process complete. Final output saved in", final_output_file)


Selected from Block_B.xlsx: {'Customer Name': 'HICHAM HALLOU', 'Number of Rooms': 1, 'Type of Apartment': '1+0 1'}
Selected from Block_A.xlsx: {'Customer Name': 'RECEP ULUSOY', 'Number of Rooms': 1, 'Type of Apartment': '1+1 1'}
Selected from Block_C.xlsx: {'Customer Name': 'SEHER YALÇIN', 'Number of Rooms': 2, 'Type of Apartment': '2+1 1'}
Selected from Block_C.xlsx: {'Customer Name': 'HÜSNÜ AYDINALP', 'Number of Rooms': 2, 'Type of Apartment': '2+1 1'}
Selected from Block_A.xlsx: {'Customer Name': 'NEDİME KOCA', 'Number of Rooms': 1, 'Type of Apartment': '1+1 1'}
Selected from Block_B.xlsx: {'Customer Name': 'HICHAM HALLOU', 'Number of Rooms': 1, 'Type of Apartment': '1+0 1'}
Selected from Block_B.xlsx: {'Customer Name': 'BAHADIR TIKNAS', 'Number of Rooms': 1, 'Type of Apartment': '1+0 1'}
Selected from Block_B.xlsx: {'Customer Name': 'AZİZ UĞUR ÇAĞLAR', 'Number of Rooms': 1, 'Type of Apartment': '1+0 1'}
Selected from Block_A.xlsx: {'Customer Name': 'ERDOĞAN KARALI', 'Number of Room

In [ ]:
import pandas as pd

# Step 1: Load the Excel files
group_a = pd.read_excel("/content/Block_A.xlsx")
group_b = pd.read_excel("/content/Block_B.xlsx")

# Step 2: Keep first 141 rows in Group_A and move extra rows to Group_B
group_a_141 = group_a.iloc[:141]
extra_rows = group_a.iloc[141:]

# Step 3: Append extra rows to Group_B
group_b_updated = pd.concat([group_b, extra_rows], ignore_index=True)

# Step 4: Drop the existing 'FlatID' column from Group_B (if it exists)
if 'FlatID' in group_b_updated.columns:
    group_b_updated = group_b_updated.drop(columns=['FlatID'])

# Step 5: Add new FlatID column starting from 1
group_b_updated.insert(0, 'FlatID', range(1, len(group_b_updated) + 1))

# Step 6: Save the updated files
group_a_141.to_excel("Updated_Group_A.xlsx", index=False)
group_b_updated.to_excel("Updated_Group_B.xlsx", index=False)

print("✅ Updated_Group_A.xlsx and Updated_Group_B.xlsx have been saved with new FlatID column.")


✅ Updated_Group_A.xlsx and Updated_Group_B.xlsx have been saved with new FlatID column.


In [ ]:
import pandas as pd

def clean_flatid(df):
    # Remove old FlatID if exists and add new one
    if 'FlatID' in df.columns:
        df = df.drop(columns=['FlatID'])
    df.insert(0, 'FlatID', range(1, len(df) + 1))
    return df

# Step 1: Load the Excel files
group_a = pd.read_excel("/content/Block_A.xlsx")
group_b = pd.read_excel("/content/Block_B.xlsx")
group_c = pd.read_excel("/content/Block_C.xlsx")

# Step 2: Split Group A into first 140 and overflow
group_a_main = group_a.iloc[:140]
group_a_extra = group_a.iloc[140:]

# Step 3: Append extra from Group A to Group B
group_b_combined = pd.concat([group_b, group_a_extra], ignore_index=True)

# Step 4: Split Group B into first 140 and overflow
group_b_main = group_b_combined.iloc[:140]
group_b_extra = group_b_combined.iloc[140:]

# Step 5: Append extra from Group B to Group C
group_c_combined = pd.concat([group_c, group_b_extra], ignore_index=True)

# Step 6: Keep only 140 in each group
group_a_final = group_a_main
group_b_final = group_b_main
group_c_final = group_c_combined  # No limit needed unless you want to cap it too

# Step 7: Clean and add new FlatID columns
group_a_final = clean_flatid(group_a_final)
group_b_final = clean_flatid(group_b_final)
group_c_final = clean_flatid(group_c_final)

# Step 8: Save the updated Excel files
group_a_final.to_excel("Updated_Group_A.xlsx", index=False)
group_b_final.to_excel("Updated_Group_B.xlsx", index=False)
group_c_final.to_excel("Updated_Group_C.xlsx", index=False)

print("✅ Final files saved:\n- Updated_Group_A.xlsx\n- Updated_Group_B.xlsx\n- Updated_Group_C.xlsx")


✅ Final files saved:
- Updated_Group_A.xlsx
- Updated_Group_B.xlsx
- Updated_Group_C.xlsx
